In [ ]:
# Configuration
DELTA_TABLE_PATH = "abfss://datacare@onelake.dfs.fabric.microsoft.com/mobileapps.Lakehouse/Tables/att/avetrack_partitioned"
MONGODB_DATABASE = "avetrack_docdb"
MONGODB_COLLECTION = "orders"
conn_str = ""

In [ ]:
# Import libraries
from pyspark.sql.functions import *

print("✅ Libraries imported")

In [ ]:
# Read data from Delta Lake
delta_df = spark.read.format("delta").load(DELTA_TABLE_PATH)
record_count = delta_df.count()

print(f"✅ Loaded {record_count:,} records")
delta_df.select("_id", "fields.run", "serviceId", "version").show(5, truncate=False)

In [ ]:
# Auto-adjust batch size based on record count
if record_count <= 20000000:
    BATCH_SIZE = "2000"
elif record_count <= 50000000:
    BATCH_SIZE = "5000"
elif record_count <= 100000000:
    BATCH_SIZE = "8000"
else:
    BATCH_SIZE = "10000"

print(f"📦 Batch size: {BATCH_SIZE}")

In [ ]:
# Repartition for optimal parallel writes to MongoDB
# Rule: 128-256 MB per partition for best performance
# Current: 4 partitions of ~100GB each (very slow!)
# Target: 1500-2000 partitions of ~200MB each (fast parallel writes)

current_partitions = delta_df.rdd.getNumPartitions()
WRITE_PARTITIONS = 2000  # Adjust based on cluster size

print(f"⚠️  Current partitions: {current_partitions}")
print(f"🔄 Repartitioning to {WRITE_PARTITIONS} for parallel writes...")

delta_df = delta_df.repartition(WRITE_PARTITIONS)

print(f"✅ Repartitioned: Each task will write ~{record_count//WRITE_PARTITIONS:,} records")
print(f"📊 Parallel tasks: {WRITE_PARTITIONS}")

In [ ]:
# Insert data into Azure DocumentDB
if not conn_str:
    raise ValueError("❌ Connection string not set")

print(f"🚀 Inserting {record_count:,} records...")

delta_df.write.format("mongodb") \
    .mode("append") \
    .option('connection.uri', conn_str) \
    .option("database", MONGODB_DATABASE) \
    .option("collection", MONGODB_COLLECTION) \
    .option("convertJson", "true") \
    .option("idFieldList", "_id,fields.run") \
    .option("ignoreNullValues", "true") \
    .option("maxBatchSize", BATCH_SIZE) \
    .option("operationType", "update") \
    .option("ordered", "true") \
    .option("upsertDocument", "true") \
    .save()

print(f"✅ Inserted {record_count:,} records")
print(f"🗄️  Database: {MONGODB_DATABASE}.{MONGODB_COLLECTION}")